# Elaborate statistics features for Silvereye

## Dependencies imports


In [ ]:
import xarray as xr
import os
import sys
import pandas as pd
from functools import wraps
import numpy as np

import matplotlib.pyplot as plt

In [ ]:
from datetime import date
from dateutil.relativedelta import relativedelta # $ pip install python-dateutil

In [ ]:
import seaborn as sns  # noqa, pandas aware plotting library

In [ ]:
if ('SP_SRC' in os.environ):
    root_src_dir = os.environ['SP_SRC']
elif sys.platform == 'win32':
    root_src_dir = r'C:\src\csiro\stash\silverpieces'
else:
    root_src_dir = '/home/per202/src/csiro/stash/silverpieces'

pkg_src_dir = root_src_dir
sys.path.append(pkg_src_dir)

In [ ]:
from silverpieces import *

In [ ]:
from silverpieces.functions import *

In [ ]:
if ('SP_DATA' in os.environ):
    root_data_dir = os.environ['SP_DATA']
elif sys.platform == 'win32':
    root_data_dir = r'C:\data\silverpieces'
else:
    root_data_dir = '/home/per202/data/silverpieces'


In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
# the default cmap_sequential for xarray is viridis. 'RdBu' is divergent, but works better for wetness concepts
# # https://matplotlib.org/3.1.0/tutorials/colors/colormaps.html
xr.set_options(cmap_sequential='RdBu')

In [ ]:
# Can get tassie_silo_rain.nc data from https://cloudstor.aarnet.edu.au/plus/s/nj2RevvD1EUD77n
fn = os.path.join(root_data_dir, 'tassie_silo_rain.nc')

In [ ]:
tassie = xr.open_dataset(os.path.join(root_data_dir, 'tassie_silo_rain.nc'))

In [ ]:
tassie

In [ ]:
daily_rain = tassie.daily_rain

In [ ]:
daily_rain.isel(time=4300).plot()

## Use cases

### 3 year period statistic compared to all 3 years periods in the historical record

We want to be able to compare a grid of statistics for a period compared to all periods of similar lengths.
The start and end of the period should be as arbitrary as possible. The sliding window could however be limited or fixed to a year: it is probably moot to compare windows with shifted seasonality. 

#### How does the cumulated rainfall 2016-2018 over TAS compare with all 3 year periods over the record?


In [ ]:
s = SpatialTemporalDataArrayStat()
start_time = pd.to_datetime('2016-01-01')
end_time = pd.to_datetime('2018-12-31')

In [ ]:
three_years_rains = s.periods_stat_yearly(daily_rain, start_time, end_time, func = np.sum)

In [ ]:
three_years_rains.name = 'cum_rain_3yr'

In [ ]:
TIME_DIMNAME = 'time'

In [ ]:
g_simple = three_years_rains.plot(x='lon', y='lat', col='time', col_wrap=3)

In [ ]:
q = np.array([.1, .5, .9])

In [ ]:
y = s.quantile_over_time_dim(three_years_rains, q=q)

In [ ]:
y.name = '3-yr cumulated rain quantiles'

The following color scheme may not be the best to see the map of quantile values, but should give an idea 

In [ ]:
y.plot(x='lon', y='lat', col='quantile', col_wrap=1, cmap='gist_ncar')

So now we want a map that tells us where the last three years, for every grid cell, sits (which side of every quantile)

In [ ]:
last_three_years_cumrain = three_years_rains[-1,:,:]

In [ ]:
cat_q = s.searchsorted(y, last_three_years_cumrain)
cat_q.name = 'Quantile categories 10/50/90'

In [ ]:
cat_q.plot(cmap='bwr_r')

So, the three years 2016 to 2018 have been the wettest on most of the mountainous areas of the state compared to the last decade, except for the south west National Park which has been the driest comparatively.